In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from google import genai
from google.genai import types
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr

In [ ]:
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GEMINI_API_KEY'] = os.getenv('GEMINI_API_KEY', 'your-key-if-not-using-env')

In [ ]:
openai = OpenAI()
claude = anthropic.Anthropic()
gemini = genai.Client()

OPENAI_MODEL = "gpt-4o"
GEMINI_MODEL = "gemini-2.5-flash"
CLAUDE_MODEL = "claude-sonnet-4-5-20250929"

In [ ]:
system_message = "You are an assistant that helps to add comment to any kind of code e.g Python, Java, C++ etc. "
system_message += "Respond with the original code plus comments in the language syntax. The comment you add to the code should not be unnecessarily verbose "
system_message += "and it should be aimed at helping the reader understand the purpose of the code and it's functionalities."

In [ ]:
def user_prompt_for(code):
    user_prompt = "Detect the language in which the provided code in written in and add comments to the code wherever necessary."
    user_prompt += "Respond with the original code plus any additional useful comment you add. Comment should be aimed at improving the understanding of the code.\n\n"
    user_prompt += code
    return user_prompt

In [ ]:
def prompt_message(code):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(code)}
    ] 

In [ ]:
def comment_code_gpt(code):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=prompt_message(code), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply

def comment_code_claude(code):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(code)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply
    
def comment_code_gemini(code):
    response = gemini.models.generate_content_stream(
        model=GEMINI_MODEL,
        config=types.GenerateContentConfig(system_instruction=system_message),
        contents=user_prompt_for(code)
        )
    reply = ""
    for chunk in response:
        reply += chunk.text
        yield reply

def comment_code(model, code):
    if model=="GPT":
        result = comment_code_gpt(code)
    elif model=="Claude":
        result = comment_code_claude(code)
    elif model=="Gemini":
        result = comment_code_gemini(code)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far    


In [ ]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

python_hard = """

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

n = 10000        
initial_seed = 42 
min_val = -10     
max_val = 10      

import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""


In [28]:
def select_sample_program(sample_program):
    if sample_program=="pi":
        return pi
    elif sample_program=="python_hard":
        return python_hard
    else:
        return "Type your Program here"

In [29]:
with gr.Blocks() as ui:
    gr.Markdown("## Add comment for any kind of code")
    with gr.Row():
        original_code = gr.Textbox(label="Code:", value=python_hard, lines=10)
        commented_code = gr.Textbox(label="Commented code", lines=10)
    with gr.Row():
        sample_program = gr.Radio(["pi", "python_hard", "custom"], label="Sample program", value="python_hard")
        model = gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select model", value="Gemini")
    with gr.Row():
        comment_code_button = gr.Button("Add comment to code")

    sample_program.change(select_sample_program, inputs=[sample_program], outputs=[original_code])
    comment_code_button.click(comment_code, inputs=[model, original_code], outputs=[commented_code])
    
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
